In [1]:
import requests
import pandas as pd
from apscheduler.schedulers.blocking import BlockingScheduler

In [2]:
def get_historic_price(symbol, exchange='coinbase', after='2021-05-01'):
    url = 'https://api.cryptowat.ch/markets/{exchange}/{symbol}usdt/ohlc'.format(symbol=symbol, exchange=exchange)
    resp = requests.get(url, params={'periods': time_interval, 'after': str(int(pd.Timestamp(after).timestamp()))})
    resp.raise_for_status()
    data = resp.json()
    df = pd.DataFrame(data['result'][time_interval], columns=['CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 
                                                              'ClosePrice', 'Volume', 'NA'])
    df['CloseTime'] = pd.to_datetime(df['CloseTime'], unit='s')
    df.set_index('CloseTime', inplace=True)
    return df

In [3]:
time_interval = '3600'  
time_length = 1          
def get_price_change():
    from_when = (pd.Timestamp.now() - pd.offsets.Day(time_length))
    btc = get_historic_price('btc', 'hitbtc', after=from_when)
    btc = btc.drop(['NA'], axis=1)
    btc_perc = btc.pct_change()*100
    df = pd.merge(left=btc, right=btc_perc, left_index=True, right_index=True)
    df = df.rename(columns={"OpenPrice_y":"OpenPrice_%", "HighPrice_y":"HighPrice_%", 
                            "LowPrice_y":"LowPrice_%", "ClosePrice_y":"ClosePrice_%", "Volume_y":"Volume_%"})
    low_change = df.iloc[-1]['LowPrice_%'] 
    return low_change

In [ ]:
def btc_panic_bot():
    low_change = get_price_change()
    if -3 <low_change <= -1:
        print('HODL.')
    elif -5 <low_change <= -3:
        print('HOOOOOOODL.')
    elif -7 <low_change <= -5:
        print('HOOOOOOOOOOOOOOOOOOOOODL!!!')
    elif low_change <= -7:
        print('HOOOOOOOOOOOOOOOODL...FUUUUUUUUUUUUUUUUUUUUUUC...')

scheduler = BlockingScheduler()
scheduler.add_job(btc_panic_bot, 'interval', minutes=1)
scheduler.start()

HOOOOOOODL.
HOOOOOOODL.
HOOOOOOODL.
HOOOOOOODL.
